In [55]:
import pandas as pd 
import numpy as np
import os

curr_directory = os.getcwd()

# Player Roster Information (Copied over from player_roster.ipynb)
teams = ['BOS','BRK','NYK','PHI','TOR','CHI','CLE','DET','IND','MIL','ATL','CHO','MIA','ORL','WAS',
         'DEN','MIN','OKC','POR','UTA','GSW','LAC','LAL','PHO','SAC','DAL','HOU','MEM','NOP','SAS']
        
# Dictionary of roster
# Ex. The roster of Boston Celtics players for the 2019-2020 season can be accessed using roster['BOS']['2019']
# It does not include any players/rookies for which there is no season data
roster = {}
    
for team in teams: 
    roster[team] = {}

# Initialize set for list of all players (with no repeats)
all_players = set()
    
for filename in os.listdir(os.path.join(curr_directory, 'data_sets/player_roster')):
    data = pd.read_csv(os.path.join('data_sets/player_roster', filename))
    year = filename[0:4]
    
    for team in teams:
        roster[team][year] = []
        
        players = data.loc[data['Tm'] == team]
        for ind in players.index: 
            player_name = players['Player'][ind].split('\\', 1)[0]
            if player_name not in roster[team][year]: 
                roster[team][year].append(player_name)
            
        all_players.update(roster[team][year])

num_players = len(all_players)
        
# Player dictionary that maps all players to index
player_index = dict(zip(list(all_players), range(len(all_players))))

print(roster['PHI']['2019'])

['Jonah Bolden', 'Trey Burke', 'Joel Embiid', 'James Ennis', 'Tobias Harris', 'Al Horford', 'Furkan Korkmaz', 'Shake Milton', 'Raul Neto', "Kyle O'Quinn", 'Josh Richardson', 'Mike Scott', 'Ben Simmons', 'Matisse Thybulle']


In [56]:
# playbyplay_Data = pd.read_csv('/Users/baiyangwang/Box Sync/academics/maching learning/final-project/NBA-PBP-2018-2019.csv')

### What is the first column being loaded here? 
game_data = pd.read_csv(os.path.join(curr_directory,'data_sets/nba.games.stats.csv'))

# Sort all values by the Date
game_data = game_data.sort_values(by=['Date'])

In [57]:
# game has chronical order and Y shows score differential, X plus is team 1 payer
# X minus is team 2 player
game_results = np.array(list(game_data['TeamPoints'] - game_data['OpponentPoints']))
teams = np.array(list(zip(game_data.Team, game_data.Opponent)))
dates = np.array(list(game_data['Date']))

unique_dates = list(set(dates))

In [58]:
# Makes an index of all games that are repeated
repeat_indexes = []

for date in unique_dates: 
    same_day = np.where(dates == date)
    # suppose same_day = [0, 1, 2, 3, 4, 5]
    for i in same_day[0]: 
        # start with i = 0
        for j in same_day[0]: 
            # j = 0, 1, 2, 3, 4, 5
            if j > i: 
                if np.array_equal(np.flip(teams[j], axis=0) , teams[i]): 
                    repeat_indexes.append(j)

# Make new unique game results, teams and dates arrays
unique_game_results = game_results[repeat_indexes]
unique_teams = teams[repeat_indexes]
unique_dates = dates[repeat_indexes]

['PHI' 'ORL']


In [53]:
print(unique_teams.shape)

(4920, 2)


In [59]:
# Initialize synergy and anti-synergy parameters
synergy = np.zeros((num_players,num_players))
antisynergy = np.zeros((num_players,num_players))

In [60]:
def x_for_game(teams, date): 
    x_1 = np.zeros((num_players, 1))
    x_2 = np.zeros((num_players, 1))

    if int(date[5:7]) < 9: 
        year = str(int(date[0:4]) - 1)
    else: 
        year = date[0:4]

    team_1_players = roster[team][year]
    for item in team_1_players: 
        x_1[player_index[item]] = 1

    team_2_players = roster[team][year]
    for item in team_2_players: 
        x_2[player_index[item]] = -1
        
    return x_1, x_2

In [61]:
def process_data(teams, dates, results): 
    num_games = teams.shape[0]

    # Create x for all games
    # To access x for 0th game -- x[:, 0] 
    x = np.zeros((num_games, 2*num_players))
    for i in range(num_games): 
        z, t = x_for_game(teams[i], dates[i])
        combined = np.vstack((z, t))
        x[i, :] = combined[:,0]
    print(x.shape)

    # Create y for all games (if team A wins, y = 1; if team B wins, y = 0)
    y = np.zeros((num_games, 1))
    for i in range(num_games): 
        if results[i] > 0: 
            y[i] = 1
        else:
            y[i] = 0

    return x, y
                
x, y = process_data(unique_teams, unique_dates, unique_game_results)

(4920, 1926)


In [63]:
print(np.where(x[5, :]==1))

(array([ 48,  54, 187, 208, 246, 297, 393, 454, 456, 527, 628, 672, 682,
       683, 800, 835, 902]),)


In [103]:
# create a class for linear regression
class LinearRegression:
    def __init__(self, step_size=0.01, max_iter=1000000, eps=1e-5, theta=None, 
                  verbose=True):
        
        self.theta = theta
        self.step_size = step_size
        self.max_iter = max_iter
        self.eps = eps

    def predict(self, teams, date):
        x, z = x_for_game(teams, date)
        team_1_S = np.matmul(np.transpose(x), np.matmul(self.synergy, x))
        team_2_S = np.matmul(np.transpose(z), np.matmul(self.synergy, z))
        cross_term = np.matmul(np.transpose(x), np.matmul(self.antisynergy, z))
        
        return team_1_S - team_2_S + 2*cross_term
    
    def gradLossFunction(self, x_teams, y_teams):
        update = 0
        for i in range(x_teams.shape[0]):
            x = np.matrix(x_teams[i,:])
            y = np.matrix(y_teams[i])
            theta = np.matrix(self.theta)   
            update += x.T*x*theta*x.T*x + x.T*y*x
            print(np.where(update > 0))
            
        return update
    
    def fit(self, x, y):
        update = 0
        if self.theta is None: 
            self.theta = np.zeros((2*num_players, 2*num_players))
        
        while update == 0 or update >= self.eps:
            update = self.step_size*self.gradLossFunction(x, y)
            self.theta = self.theta - update
            print(update)
        
    def process_data(self, teams, dates, results): 
        num_games = teams.shape[0]

        # Create x for all games
        # To access x for 0th game -- x[:, 0] 
        x = np.zeros((num_games, 2*num_players))
        for i in range(num_games): 
            z, t = x_for_game(teams[i], dates[i])
            combined = np.vstack((z, t))
            x[i, :] = combined[:,0]

        # Create y for all games (if team A wins, y = 1; if team B wins, y = 0)
        y = np.zeros((num_games, 1))
        for i in range(num_games): 
            if results[i] > 0: 
                y[i] = 1
            else:
                y[i] = 0
                
        return x, y

    def x_for_game(self, teams, date): 
        x_1 = np.zeros((num_players, 1))
        x_2 = np.zeros((num_players, 1))

        if int(date[5:7]) < 9: 
            year = str(int(date[0:4]) - 1)
        else: 
            year = date[0:4]

        team_1_players = roster[team][year]
        for item in team_1_players: 
            x_1[player_index[item]] = 1

        team_2_players = roster[team][year]
        for item in team_2_players: 
            x_2[player_index[item]] = -1

        return x_1, x_2

In [104]:
test = LinearRegression()
x, y = test.process_data(unique_teams, unique_dates, unique_game_results)
test.fit(x, y)

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([  48,   48,   48,   48,   48,   48,   48,   48,   48,   48,   48,
         48,   48,   48,   48,   48,   48,   54,   54,   54,   54,   54,
         54,   54,   54,   54,   54,   54,   54,   54,   54,   54,   54,
         54,  187,  187,  187,  187,  187,  187,  187,  187,  187,  187,
        187,  187,  187,  187,  187,  187,  187,  208,  208,  208,  208,
        208,  208,  208,  208,  208,  208,  208,  208,  208,  208,  208,
        208,  208,  246,  246,  246,  246,  246,  246,  246,  246,  246,
        246,  246,  246,  246,  246,  246,  246,  246,  297,  297,  297,
        297,  297,  297,  297,  297,  297,  297,  297,  297,  297,  297,
        297,  297,  297,  393,  393,  393,  393,  393,  393,  393,  393,
        393,  393,  393,  393,  393,  393,  393,  393,  393,  454,  454,
        454,  454,  454,  454,  454,  454,  454,  454,  454,  454,  454,
        454,  454,  454, 

(array([  48,   48,   48,   48,   48,   48,   48,   48,   48,   48,   48,
         48,   48,   48,   48,   48,   48,   54,   54,   54,   54,   54,
         54,   54,   54,   54,   54,   54,   54,   54,   54,   54,   54,
         54,  187,  187,  187,  187,  187,  187,  187,  187,  187,  187,
        187,  187,  187,  187,  187,  187,  187,  208,  208,  208,  208,
        208,  208,  208,  208,  208,  208,  208,  208,  208,  208,  208,
        208,  208,  246,  246,  246,  246,  246,  246,  246,  246,  246,
        246,  246,  246,  246,  246,  246,  246,  246,  297,  297,  297,
        297,  297,  297,  297,  297,  297,  297,  297,  297,  297,  297,
        297,  297,  297,  393,  393,  393,  393,  393,  393,  393,  393,
        393,  393,  393,  393,  393,  393,  393,  393,  393,  454,  454,
        454,  454,  454,  454,  454,  454,  454,  454,  454,  454,  454,
        454,  454,  454,  454,  456,  456,  456,  456,  456,  456,  456,
        456,  456,  456,  456,  456,  456,  456,  

(array([  48,   48,   48,   48,   48,   48,   48,   48,   48,   48,   48,
         48,   48,   48,   48,   48,   48,   54,   54,   54,   54,   54,
         54,   54,   54,   54,   54,   54,   54,   54,   54,   54,   54,
         54,  187,  187,  187,  187,  187,  187,  187,  187,  187,  187,
        187,  187,  187,  187,  187,  187,  187,  208,  208,  208,  208,
        208,  208,  208,  208,  208,  208,  208,  208,  208,  208,  208,
        208,  208,  246,  246,  246,  246,  246,  246,  246,  246,  246,
        246,  246,  246,  246,  246,  246,  246,  246,  297,  297,  297,
        297,  297,  297,  297,  297,  297,  297,  297,  297,  297,  297,
        297,  297,  297,  393,  393,  393,  393,  393,  393,  393,  393,
        393,  393,  393,  393,  393,  393,  393,  393,  393,  454,  454,
        454,  454,  454,  454,  454,  454,  454,  454,  454,  454,  454,
        454,  454,  454,  454,  456,  456,  456,  456,  456,  456,  456,
        456,  456,  456,  456,  456,  456,  456,  

(array([  48,   48,   48,   48,   48,   48,   48,   48,   48,   48,   48,
         48,   48,   48,   48,   48,   48,   54,   54,   54,   54,   54,
         54,   54,   54,   54,   54,   54,   54,   54,   54,   54,   54,
         54,  187,  187,  187,  187,  187,  187,  187,  187,  187,  187,
        187,  187,  187,  187,  187,  187,  187,  208,  208,  208,  208,
        208,  208,  208,  208,  208,  208,  208,  208,  208,  208,  208,
        208,  208,  246,  246,  246,  246,  246,  246,  246,  246,  246,
        246,  246,  246,  246,  246,  246,  246,  246,  297,  297,  297,
        297,  297,  297,  297,  297,  297,  297,  297,  297,  297,  297,
        297,  297,  297,  393,  393,  393,  393,  393,  393,  393,  393,
        393,  393,  393,  393,  393,  393,  393,  393,  393,  454,  454,
        454,  454,  454,  454,  454,  454,  454,  454,  454,  454,  454,
        454,  454,  454,  454,  456,  456,  456,  456,  456,  456,  456,
        456,  456,  456,  456,  456,  456,  456,  

(array([  48,   48,   48,   48,   48,   48,   48,   48,   48,   48,   48,
         48,   48,   48,   48,   48,   48,   54,   54,   54,   54,   54,
         54,   54,   54,   54,   54,   54,   54,   54,   54,   54,   54,
         54,  187,  187,  187,  187,  187,  187,  187,  187,  187,  187,
        187,  187,  187,  187,  187,  187,  187,  208,  208,  208,  208,
        208,  208,  208,  208,  208,  208,  208,  208,  208,  208,  208,
        208,  208,  246,  246,  246,  246,  246,  246,  246,  246,  246,
        246,  246,  246,  246,  246,  246,  246,  246,  297,  297,  297,
        297,  297,  297,  297,  297,  297,  297,  297,  297,  297,  297,
        297,  297,  297,  393,  393,  393,  393,  393,  393,  393,  393,
        393,  393,  393,  393,  393,  393,  393,  393,  393,  454,  454,
        454,  454,  454,  454,  454,  454,  454,  454,  454,  454,  454,
        454,  454,  454,  454,  456,  456,  456,  456,  456,  456,  456,
        456,  456,  456,  456,  456,  456,  456,  

...

(array([  48,   48,   48,   48,   48,   48,   48,   48,   48,   48,   48,
         48,   48,   48,   48,   48,   48,   54,   54,   54,   54,   54,
         54,   54,   54,   54,   54,   54,   54,   54,   54,   54,   54,
         54,  187,  187,  187,  187,  187,  187,  187,  187,  187,  187,
        187,  187,  187,  187,  187,  187,  187,  208,  208,  208,  208,
        208,  208,  208,  208,  208,  208,  208,  208,  208,  208,  208,
        208,  208,  246,  246,  246,  246,  246,  246,  246,  246,  246,
        246,  246,  246,  246,  246,  246,  246,  246,  286,  286,  286,
        286,  286,  286,  286,  286,  286,  286,  286,  286,  286,  286,
        286,  286,  286,  297,  297,  297,  297,  297,  297,  297,  297,
        297,  297,  297,  297,  297,  297,  297,  297,  297,  297,  297,
        297,  297,  297,  297,  297,  297,  306,  306,  306,  306,  306,
        306,  306,  306,  306,  306,  306,  306,  306,  306,  306,  306,
        306,  393,  393,  393,  393,  393,  393,  

(array([  48,   48,   48, ..., 1865, 1865, 1865]), array([  48,   54,  187, ..., 1763, 1798, 1865]))
(array([  48,   48,   48, ..., 1865, 1865, 1865]), array([  48,   54,  187, ..., 1763, 1798, 1865]))
(array([  48,   48,   48, ..., 1865, 1865, 1865]), array([  48,   54,  187, ..., 1763, 1798, 1865]))
(array([  48,   48,   48, ..., 1865, 1865, 1865]), array([  48,   54,  187, ..., 1763, 1798, 1865]))
(array([  48,   48,   48, ..., 1865, 1865, 1865]), array([  48,   54,  187, ..., 1763, 1798, 1865]))
(array([  48,   48,   48, ..., 1865, 1865, 1865]), array([  48,   54,  187, ..., 1763, 1798, 1865]))
(array([  48,   48,   48, ..., 1865, 1865, 1865]), array([  48,   54,  187, ..., 1763, 1798, 1865]))
(array([  48,   48,   48, ..., 1865, 1865, 1865]), array([  48,   54,  187, ..., 1763, 1798, 1865]))
(array([  48,   48,   48, ..., 1865, 1865, 1865]), array([  48,   54,  187, ..., 1763, 1798, 1865]))
(array([  48,   48,   48, ..., 1865, 1865, 1865]), array([  48,   54,  187, ..., 1763, 1798

(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798

(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798, 1903]))
(array([  48,   48,   48, ..., 1903, 1903, 1903]), array([  48,   54,  187, ..., 1659, 1798

KeyboardInterrupt: 